# What's in this exercise?
We run the common functions notebook so we can reuse capability defined there, and then...<BR>
1) Load yellow taxi data in staging directory to raw data directory, and save as parquet<BR> 
2) Create external unmanaged Hive tables<BR>
3) Create statistics for tables

In [221]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,LongType,FloatType,DoubleType, TimestampType

In [222]:
storageAccountName = dbutils.secrets.get(scope="azure-databricks", key="storage-name")
storageAccountKey = dbutils.secrets.get(scope="azure-databricks", key="storage-key")

In [223]:
spark.conf.set(f"fs.azure.account.key.{storageAccountName}.dfs.core.windows.net", storageAccountKey)

In [224]:
# Define source and destination directories
srcDataDirRoot = f"abfss://bronze@{storageAccountName}.dfs.core.windows.net/transactional-data/" #Root dir for source data
destDataDirRoot = f"abfss://curated@{storageAccountName}.dfs.core.windows.net/nyctaxi/transactions/yellow-taxi/" #Root dir for consumable data

#Canonical ordered column list for yellow taxi across years to homogenize schema
canonicalTripSchemaColList = ["taxi_type","vendor_id","pickup_datetime","dropoff_datetime","store_and_fwd_flag","rate_code_id","pickup_location_id","dropoff_location_id","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count","trip_distance","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","payment_type","trip_year","trip_month"]

### 1.  Execute notebook with common/reusable functions

In [225]:
%run "../../01-General/2-CommonFunctions.ipynb"

Code from file 'file:///home/dinhnn/git/personal/databricks/pocbigdata/Workspace/01-General/2-CommonFunctions.ipynb':
 import os

import math

import glob

import re
prqShrinkageFactor = 0.19 #We found a saving in space of 81% with Parquet
def analyzeTables(databaseAndTable):

  try:

    print("Table: " + databaseAndTable)

    print("....refresh table")

    sql("REFRESH TABLE " + databaseAndTable)

    print("....analyze table")

    sql("ANALYZE TABLE " + databaseAndTable + " COMPUTE STATISTICS")

    print("....done")

  except Exception as e:

    return e
def calcOutputFileCountTxtToPrq(srcDataFile, targetedFileSizeMB):

  try:

    estFileCount = int(math.floor((os.path.getsize(srcDataFile) * prqShrinkageFactor) / (targetedFileSizeMB * 1024 * 1024)))

    if(estFileCount == 0):

      return 1 

    else:

      return estFileCount

  except Exception as e:

    return e
#Delete residual files from job operation (_SUCCESS, _start*, _committed*)

#Should be called with '/dbfs/mn

#### 2. Define schema for source data
Different years have different schemas - fields added/removed

In [226]:
#Schema for data based on year and month

# 2017
yellowTripSchema2017H1 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

#Second half of 2016
yellowTripSchema2016H2 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("junk1", StringType(), True),
    StructField("junk2", StringType(), True)])

# 2015 and 2016 first half of the year
yellowTripSchema20152016H1 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

# 2009 though 2014
yellowTripSchemaPre2015 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

#### 3. Some functions

In [227]:
#1) Function to determine schema for a given year and month
#Input:  Year and month
#Output: StructType for applicable schema 
#Sample call: print getSchemaStruct(2009,1)

def getTaxiSchema(tripYear, tripMonth):
  taxiSchema = None

  if(tripYear > 2008 and tripYear < 2015):
    taxiSchema = yellowTripSchemaPre2015
  elif(tripYear == 2016 and tripMonth > 6):
    taxiSchema = yellowTripSchema2016H2
  elif((tripYear == 2016 and tripMonth < 7) or (tripYear == 2015)):
    taxiSchema = yellowTripSchema20152016H1
  elif(tripYear == 2017 and tripMonth < 7):
    taxiSchema = yellowTripSchema2017H1
  
  return taxiSchema

In [228]:
#2) Function to add columns to dataframe as required to homogenize schema
#Input:  Dataframe, year and month
#Output: Dataframe with homogenized schema 
#Sample call: println(getSchemaHomogenizedDataframe(DF,2014,6))

from pyspark.sql.functions import *

def getSchemaHomogenizedDataframe(sourceDF,tripYear, tripMonth):
  if(tripYear > 2008 and tripYear < 2015):
    sourceDF = (sourceDF.withColumn("pickup_location_id", lit(0).cast("integer"))
              .withColumn("dropoff_location_id", lit(0).cast("integer"))
              .withColumn("improvement_surcharge",lit(0).cast("double"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_pickup_longitude", col("pickup_longitude").cast("string"))
                                      .drop("pickup_longitude").withColumnRenamed("temp_pickup_longitude", "pickup_longitude")
              .withColumn("temp_dropoff_longitude", col("dropoff_longitude").cast("string"))
                                      .drop("dropoff_longitude").withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")
              .withColumn("temp_pickup_latitude", col("pickup_latitude").cast("string"))
                                      .drop("pickup_latitude").withColumnRenamed("temp_pickup_latitude", "pickup_latitude")
              .withColumn("temp_dropoff_latitude", col("dropoff_latitude").cast("string"))
                                      .drop("dropoff_latitude").withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif((tripYear == 2016 and tripMonth < 7) or (tripYear == 2015)):
    sourceDF = (sourceDF.withColumn("pickup_location_id", lit(0).cast("integer"))
              .withColumn("dropoff_location_id", lit(0).cast("integer"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_pickup_longitude", col("pickup_longitude").cast("string"))
                                      .drop("pickup_longitude").withColumnRenamed("temp_pickup_longitude", "pickup_longitude")
              .withColumn("temp_dropoff_longitude", col("dropoff_longitude").cast("string"))
                                      .drop("dropoff_longitude").withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")
              .withColumn("temp_pickup_latitude", col("pickup_latitude").cast("string"))
                                      .drop("pickup_latitude").withColumnRenamed("temp_pickup_latitude", "pickup_latitude")
              .withColumn("temp_dropoff_latitude", col("dropoff_latitude").cast("string"))
                                      .drop("dropoff_latitude").withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif(tripYear == 2016 and tripMonth > 6):
    sourceDF = (sourceDF.withColumn("pickup_longitude", lit(""))
              .withColumn("pickup_latitude", lit(""))
              .withColumn("dropoff_longitude", lit(""))
              .withColumn("dropoff_latitude", lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif(tripYear == 2017 and tripMonth < 7):
    sourceDF = (sourceDF.withColumn("pickup_longitude", lit(""))
              .withColumn("pickup_latitude", lit(""))
              .withColumn("dropoff_longitude", lit(""))
              .withColumn("dropoff_latitude", lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  else:
    sourceDF
    
  return sourceDF

#### 4. Read CSV, homogenize schema across years, save as parquet

In [ ]:

# To make Hive Parquet format compatible with Spark Parquet format
spark.conf.set("spark.sql.parquet.writeLegacyFormat", "true")

# Disable delta optimizeWrite for consistent number of partition (when write from 2nd time, spark with delta format can reduce number of partitions)
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "false")

#Process data, save as parquet
for j in range(2009,2018):
    
  endMonth = None
  if (j==2017):
    endMonth = 6 
  else: endMonth = 12
  
  for i in range(1,endMonth+1):
    
    srcDataFile= "{}year={}/month={:02d}/type=yellow/yellow_tripdata_{}-{:02d}.csv".format(srcDataDirRoot,j,i,j,i)
    print("Year={}; Month={}".format(j,i))
    print(srcDataFile)

    #Source schema
    taxiSchema = getTaxiSchema(j,i)

    #Read source data
    taxiDF = (spark.read.format("csv")
                    .option("header", True)
                    .schema(taxiSchema)
                    .option("delimiter",",")
                    .load(srcDataFile).cache())


    #Add additional columns to homogenize schema across years
    taxiFormattedDF = getSchemaHomogenizedDataframe(taxiDF, j, i)

    #Order all columns to align with the canonical schema for yellow taxi
    taxiCanonicalDF = taxiFormattedDF.select(*canonicalTripSchemaColList)
    
    year_month_dir = "{}trip_year={}/trip_month={:02d}".format(destDataDirRoot,j,i)
    dbutils.fs.rm(year_month_dir,recurse=True)

    taxiCanonicalDF.repartition(8).write.option("compression", "zstd").format("delta").mode("append").partitionBy("trip_year","trip_month").save(destDataDirRoot)  

Year=2009; Month=1
abfss://bronze@[REDACTED].dfs.core.windows.net/transactional-data/year=2009/month=01/type=yellow/yellow_tripdata_2009-01.csv
Year=2009; Month=2
abfss://bronze@[REDACTED].dfs.core.windows.net/transactional-data/year=2009/month=02/type=yellow/yellow_tripdata_2009-02.csv
Year=2009; Month=3
abfss://bronze@[REDACTED].dfs.core.windows.net/transactional-data/year=2009/month=03/type=yellow/yellow_tripdata_2009-03.csv

In [245]:
def create_table(schema: str, table_name: str, parquet_dir: str, location: str):
    spark.sql(f"use {schema};")
    spark.sql(f"DROP TABLE IF EXISTS {table_name};")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING delta LOCATION '{location}/{parquet_dir}';")

In [246]:
# Function to convert Spark schema to SQL DDL format
def convert_schema_to_sql_ddl(schema):
    """
    Convert a Spark StructType schema to SQL DDL format.
    
    Parameters:
    - schema: A StructType schema
    
    Returns:
    - A string with SQL DDL column definitions in parentheses
    
    Example:
    StructType([
        StructField("location_id", StringType(), True),
        StructField("borough", StringType(), True)
    ])
    
    Returns:
    (
        location_id STRING,
        borough STRING
    )
    """
    column_definitions = []
    for field in schema.fields:
        field_type = field.dataType.simpleString()
        print(f"field = {field}, field_type = {field_type}")
        # Map Spark types to SQL types
        if field_type == "string":
            sql_type = "STRING"
        elif field_type in ("integer", "int"):
            sql_type = "INT"
        elif field_type == "long":
            sql_type = "BIGINT"
        elif field_type == "double":
            sql_type = "DOUBLE"
        elif field_type == "float":
            sql_type = "FLOAT"
        elif field_type == "boolean":
            sql_type = "BOOLEAN"
        elif field_type == "timestamp":
            sql_type = "TIMESTAMP"
        elif field_type == "date":
            sql_type = "DATE"
        elif field_type.startswith("decimal"):
            sql_type = field_type.upper()
        else:
            sql_type = "STRING"  # Default to STRING for unknown types
        
        column_definitions.append(f"        {field.name} {sql_type}")
    
    # Format as a parenthesized list
    return "(\n" + ",\n".join(column_definitions) + "\n)"

In [247]:
def create_table(schema: str, table_name: str, parquet_dir: str, location: str):
    spark.sql(f"use {schema};")
    spark.sql(f"DROP TABLE IF EXISTS {table_name};")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING delta LOCATION '{location}/{parquet_dir}';")

In [248]:
create_table(schema="synapse_nyc_reference.nyctaxi", table_name="yellow_taxi_trips_raw", parquet_dir="", location=destDataDirRoot)

In [249]:
%sql
FSCK REPAIR TABLE nyctaxi.yellow_taxi_trips_raw;

<Empty result set>

#### 5. Statistics table

In [80]:
%sql
select * from nyctaxi.yellow_taxi_trips_raw;

taxi_type,vendor_id,pickup_datetime,dropoff_datetime,store_and_fwd_flag,rate_code_id,pickup_location_id,dropoff_location_id,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_year,trip_month
yellow,VTS,2009-02-05T18:12:00Z,2009-02-05T18:14:00Z,null,null,0,0,-73.937792,40.757263,-73.935107,40.759568,5,2.22,8.5,1,null,2,0,0,11.5,Credit,2009,02
yellow,VTS,2009-02-11T13:57:00Z,2009-02-11T14:05:00Z,null,null,0,0,-73.965687,40.762827,-73.955135,40.764912,1,0.79,5.7,0,null,0,0,0,5.7,CASH,2009,02
yellow,CMT,2009-02-15T00:43:09Z,2009-02-15T00:50:24Z,0,null,0,0,-74.007475,40.740969,-74.004874,40.751528,1,1.4,7.4,0,null,0,0,0,7.4,Cash,2009,02
yellow,VTS,2009-02-06T21:39:00Z,2009-02-06T21:55:00Z,null,null,0,0,-73.862655,40.768917,-73.973753,40.753232,5,9.33,21.3,0.5,null,0,4.15,0,25.95,CASH,2009,02
yellow,CMT,2009-02-04T10:59:38Z,2009-02-04T11:01:21Z,null,null,0,0,-73.952474,40.772097,-73.953406,40.76664,1,0.4,3.3,0,null,0,0,0,3.3,Cash,2009,02
yellow,VTS,2009-02-08T19:57:00Z,2009-02-08T20:09:00Z,null,null,0,0,-74.016098,40.709983,-73.990048,40.767095,5,4.37,12.1,0,null,3,0,0,15.1,Credit,2009,02
yellow,CMT,2009-02-04T21:23:16Z,2009-02-04T21:26:02Z,null,null,0,0,-73.976372,40.763171,-73.986303,40.768557,1,0.5,4.2,0,null,0,0,0,4.2,Cash,2009,02
yellow,CMT,2009-02-11T20:12:38Z,2009-02-11T20:24:10Z,0,null,0,0,-73.953333,40.78677,-73.954212,40.764146,1,2.2,9.4,0,null,0,0,0,9.4,Cash,2009,02
yellow,CMT,2009-02-11T07:44:07Z,2009-02-11T07:54:46Z,0,null,0,0,-73.981469,40.746767,-73.956414,40.764725,1,2.6,8.9,0,null,0,0,0,8.9,Cash,2009,02
yellow,VTS,2009-02-07T14:59:00Z,2009-02-07T15:02:00Z,null,null,0,0,-73.978203,40.783578,-73.974815,40.790593,1,0.6,4.1,0,null,0,0,0,4.1,CASH,2009,02


In [250]:
%sql
select COUNT(1) from nyctaxi.yellow_taxi_trips_raw;

count(1)
1369889765
